# Dataproject on life expectancy

Imports and set magics:

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import plotly.graph_objects as go
import ipywidgets as widgets

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


In [ ]:
#pip install geopandas

In [ ]:
#pip install linearmodels

# Our data 

In this data project we will investigate how daily smoking, social support and alcohol consumption affects life expectancy at age 65 years old. The explanatory variabels are chosen based on an expected effect. All data are taken from the OECD database (https://data.oecd.org) and are imported as csv-files. All datasets contain observations in the interval from 1960 to 2021. We import at the following datasets:

-  **lifeexp65.csv:** Life expectancy at 65 is measured as the average number of years that a person at 65 years can be expected to live assuming that age-specifikc mortality levels remain constant. The dataset contains observations for men and women individually. We also import the two datasets *men_lifeexp65.csv* and *women_lifeexp65.csv* which contain *lifeexp65* data for men and women respectively in 2020. 

-  **alcohol.csv:** Alchol consumption is defined as annual sales of pure alcohol in litres per person aged 15 years and older. The dataset contains obervations for the entire population.

-  **smokers.csv:** Daily smokers are defined as the population aged 15 years and older who are reporting to smoke every day. The indicator is measured as a percentage of the population aged 15 years and older and is stated for men, women and the total population.

-  **socsupport.csv:** Social support indicates the share of people who report having friends or relatives whom they can count on in times of trouble. The indicator is measures as a percentage of survey respondent. Due to small sample sizes, country averages are pooled between 2010 and 2020. The survey sample is designed to be nationally representative of the population ages 15 years and older. 

To give an overview of the current lifeexpectancy on a global plan we present *lifeexp65* in two maps showing life expectancy at 65 for men and women respectively, in 2020 for data in our dataset. 

In [ ]:
# making overview map for countries in data set

for data in ['men_lifeexp65.csv', 'women_lifeexp65.csv']:
    # rewrite to use in titles in final plot
    data_title = data.rstrip('_lifeexp65.csv')
    # load and clean data
    mapdata = pd.read_csv(data)
    drop = ['INDICATOR','SUBJECT','MEASURE','FREQUENCY','TIME','Flag Codes']
    mapdata.drop(drop, axis=1, inplace=True)

    # load data to make world map
    df_world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # merge map data to world map
    df_world_mapdata = df_world.merge(mapdata, how="left", left_on=['iso_a3'], right_on=['LOCATION'])

    # add visuals, axes, labels and title
    fig, ax = plt.subplots(1, 1, figsize=(20, 16))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad="0.5%")
    df_world_mapdata.plot(column="Value", ax=ax, cax=cax, cmap='OrRd',legend=True, legend_kwds={"label": "Life expectancy at 65"}, missing_kwds={'color':'lightgrey'}, vmin=5, vmax=28)
    ax.set_title(f'Life expectancy at 65 for countries in data set (2020), {data_title}')




# Read and clean data

#### We import the 4 different CSV files

In [ ]:
# importing data
alcohol = pd.read_csv('alcohol.csv')
life_exp_65 = pd.read_csv('lifeexp65.csv')
smokers = pd.read_csv('smokers.csv')
social_support = pd.read_csv('socsupport.csv')

Forsøg på at loope csv filerne ind - men hvordan gemmer hver for sig og ikke i en liste

In [ ]:
dataframes = []
filename = ['alcohol','smokers','lifeexp65','socsupport']
for i in range(len(filename)):
    temp_df = pd.read_csv(f"{filename[i]}.csv")
    dataframes.append(temp_df)

alcohol = dataframes[0]
smokers = dataframes[1]
life_exp_65 = dataframes[2]
social_support = dataframes[3]


### Cleaning each dataset

Prøver at rense alle datasættene samtidig

In [ ]:
drop_these = ['INDICATOR','MEASURE','FREQUENCY','Flag Codes'] 
for df in range(len(dataframes)) : 
   df = dataframes[i]
   df.drop(drop_these, axis=1, inplace=True)

In [4]:
for df in ['alcohol','life_exp_65','smokers','social_support']:
    df.drop(drop, axis=1, inplace=True)

AttributeError: 'str' object has no attribute 'drop'

#### Life expentancy at 65

In [ ]:
# clean life_exp_65 data
drop = ['INDICATOR','MEASURE','FREQUENCY','Flag Codes']
life_exp_65.drop(drop, axis=1, inplace=True)

life_exp_65.rename(columns={'LOCATION':'country', 'SUBJECT':'sex','TIME':'year', 'Value':'exp_years'}, inplace=True)

life_exp_65_wide = pd.pivot_table(life_exp_65,index='country',columns='year',values='exp_years')

#### Alcohol

In [ ]:
# clean alcohol data
drop = ['INDICATOR','SUBJECT','MEASURE','FREQUENCY','Flag Codes']
alcohol.drop(drop, axis=1, inplace=True)

alcohol.rename(columns={'LOCATION':'country', 'TIME':'year', 'Value':'alcohol_sale'}, inplace=True)

#### Smokers

In [ ]:
# clean smokers data
I = smokers.SUBJECT.str.contains('WOMEN')
I |= smokers.SUBJECT.str.contains('MEN')
smokers.loc[I,:]
smokers = smokers.loc[I == False]

drop = ['INDICATOR','SUBJECT','MEASURE','FREQUENCY','Flag Codes']
smokers.drop(drop, axis=1, inplace=True)

smokers.rename(columns={'LOCATION':'country', 'TIME':'year', 'Value':'smoking'}, inplace=True)

#### Social support

In [ ]:
# clean social_support data
I = social_support.SUBJECT.str.contains('TOT')
social_support.loc[I,:]
social_support = social_support.loc[I == True]

drop = ['INDICATOR','SUBJECT','MEASURE','FREQUENCY','Flag Codes']
social_support.drop(drop, axis=1, inplace=True)

social_support.rename(columns={'LOCATION':'country', 'TIME':'year', 'Value':'share_support'}, inplace=True)

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

# Merge data sets

We stack the wide dataset of life expectancy to create a long format, we can merge with the other datasets. 

In [ ]:
life_exp_stack = life_exp_65_wide.stack().reset_index()
life_exp_stack.rename(columns={0:'avg_life_exp65'},inplace=True)
life_exp_stack

In [ ]:
mergedtest = life_exp_stack.merge(alcohol, on=['country','year'],how='left').merge(smokers, on=['country','year'],how='left').merge(social_support,on=['country','year'],how='left')
mergedtest.set_index(['country','year'])

Gammel kode

In [ ]:
#merging the data

smokeralc = pd.merge(smokers,alcohol,on=['country','year'],how='left')
smokalcsoc = pd.merge(smokeralc, social_support,on=['country','year'],how='left')
merged = pd.merge(life_exp_stack, smokalcsoc ,on=['country','year'],how='left')

merged.rename(columns = {'0':'life_exp'},inplace=True)
merged.set_index(['country','year'])
merged.loc[merged.country == 'AUS'].head(40)

### KODE TIL FÆLLES WIDGET

Forsøg på ny kode med life exp og det nye mergede dataset OBS problemer

In [ ]:
def plot(df, country, var):
    I=df['country']== country
    ax=df.loc[I,:].plot(x='year', y=str(var), legend=False) 
    
def plot_timeseries(df):
    widgets.interact(plot, 
    df = widgets.fixed(df),
    var = widgets.Dropdown(
        description='variable', 
        options=['avg_life_exp65', 'alcohol_sale','smoking','share_support'], 
        value='avg_life_exp65'),
    country = widgets.Dropdown(
        description='country', 
        options=df.country.value_counts().index.to_list(), 
        value='AUS')          
);

In [ ]:
plot_timeseries(mergedtest)

Original kode

In [ ]:
def plot(df, country, var):
    I=df['country']== country
    ax=df.loc[I,:].plot(x='year', y=str(var), legend=False) 
    
def plot_timeseries(df):
    widgets.interact(plot, 
    df = widgets.fixed(df),
    var = widgets.Dropdown(
        description='variable', 
        options=['smoking','alcohol_sale','share_support'], 
        value='smoking'),
    country = widgets.Dropdown(
        description='country', 
        options=df.country.value_counts().index.to_list(), 
        value='AUS')          
);

In [ ]:
plot_timeseries(merged)

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

Vi laver inner merged til vores regression - kun år hvor hvert land har alle værdier

Ny merge kode

In [ ]:
mergedregress2= life_exp_stack.merge(alcohol, on=['country','year'],how='inner').merge(smokers, on=['country','year'],how='inner').merge(social_support,on=['country','year'],how='inner')
mergedregress2.set_index(['country','year'])

Gammel merge kode:

In [ ]:
#merging the data for regression
smokeralcregress = pd.merge(smokers,alcohol,on=['country','year'],how='inner')
smokalcsocregress = pd.merge(smokeralcregress, social_support,on=['country','year'],how='inner')
mergedregress = pd.merge(df1, smokalcsocregress ,on=['country','year'],how='inner')
mergedregress.head(20)
mergedregress.rename(columns={0:'life_exp'},inplace=True)
mergedregress.head(20)

In [ ]:
index = pd.MultiIndex.from_frame(mergedregress[['country', 'year']])

regression = pd.DataFrame(mergedregress[mergedregress.columns.to_list()[2:]].values ,index=index, columns=['y','x1','x2','x3'])

from linearmodels.panel import PanelOLS

mod = PanelOLS.from_formula("y ~ 1 + x1 + x2 + x3 + EntityEffects + TimeEffects", data=regression)

res = mod.fit(cov_type='clustered', cluster_entity=True)

print(res)

In [ ]:
res.params

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.